# E-Commerce Sales Analytics
This notebook loads the dataset, cleans it, performs EDA, and builds a basic forecast.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from dateutil.relativedelta import relativedelta

# Settings
pd.set_option('display.max_columns', 100)


In [ ]:

# Load denormalized dataset
df = pd.read_csv(r"/mnt/data/Ecommerce_Sales_Analytics/data/ecommerce_sales.csv", parse_dates=['order_date','ship_date'])
df.head()


In [ ]:

# Basic cleaning
df = df[df['revenue'] >= 0]
df['order_month'] = df['order_date'].dt.to_period('M').dt.to_timestamp()
df['year'] = df['order_date'].dt.year
df['weekday'] = df['order_date'].dt.day_name()
df.isna().sum()


In [ ]:

# KPIs
kpis = {
    'orders': df['order_id'].nunique(),
    'customers': df['customer_id'].nunique(),
    'products': df['product_id'].nunique(),
    'revenue': round(df['revenue'].sum(),2),
    'profit': round(df['profit'].sum(),2)
}
kpis


In [ ]:

# Monthly revenue trend
monthly = df.groupby('order_month').agg(revenue=('revenue','sum'), profit=('profit','sum')).reset_index()
monthly.plot(x='order_month', y='revenue', figsize=(10,4), title='Monthly Revenue'); plt.show()
monthly.tail()


In [ ]:

# Revenue by state (India focus)
geo = (df[df['country']=='India']
       .groupby(['state'])
       .agg(revenue=('revenue','sum'))
       .sort_values('revenue', ascending=False)
       .head(10))
geo


In [ ]:

# Top products
top_products = (df.groupby(['product_id','product_name','category'])
                .agg(revenue=('revenue','sum'), qty=('quantity','sum'))
                .sort_values('revenue', ascending=False)
                .head(15))
top_products


In [ ]:

# Discount impact on quantity
df['discount_bucket'] = pd.cut(df['discount'], bins=[0,0.1,0.2,0.3,0.4,0.5], include_lowest=True)
discount_impact = df.groupby('discount_bucket').agg(avg_qty=('quantity','mean'), revenue=('revenue','sum'))
discount_impact


In [ ]:

# Simple SARIMAX forecast on monthly revenue (illustrative)
series = monthly.set_index('order_month')['revenue']
train = series.iloc[:-3]
model = SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
results = model.fit(disp=False)
pred = results.get_forecast(steps=6)
forecast = pred.predicted_mean

ax = series.plot(figsize=(10,4), label='Actual')
forecast.plot(ax=ax, label='Forecast')
plt.title('Monthly Revenue Forecast')
plt.legend(); plt.show()

forecast.tail(6)
